In [22]:
from Crypto.Cipher import AES

def xor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])

def encrypt(data, key):
    nonce = b'sixteen.........'
    return AES.new(key, AES.MODE_EAX, nonce=nonce).encrypt(data)

def decrypt(ct, key):
    nonce = b'sixteen.........'
    return AES.new(key, AES.MODE_EAX, nonce=nonce).decrypt(ct)

In [23]:
# adding junk at the end so the encryptions are more syntactically dissimilar
c_0, c_1 = b'subcircuit-0..qwe', b'subcircuit-1..iop' 
s_0, s_1 = b'label-0.000....q', b'label-1.111....w'

# GEN

# encrypt c_0 with key s_1
m_0 = encrypt(c_0, key=s_1)

# encrypt c_1 with key s_0
m_1 = encrypt(c_1, key=s_0)

m_cond = xor(m_0, m_1)
m_0, m_1, m_cond

(b'7\x8b\x01\xd5v\x91\\\x147\x1b\xb1\x171\x02=,a',
 b'D\x9f]\xdd\x11\xeb\xb4\xec\x92iE\xfa.q)S+',
 b's\x14\\\x08gz\xe8\xf8\xa5r\xf4\xed\x1fs\x14\x7fJ')

In [25]:
# EVAL
S = s_1

# encrypt c_1 with the unknown key to recover m_0 or garbage
eval_m_1 = encrypt(c_1, key=S)
candidate_m_0 = xor(m_cond, eval_m_1)

# encrypt c_0 with the unknown key to recover m_1 or garbage
eval_m_0 = encrypt(c_0, key=S)
candidate_m_1 = xor(m_cond, eval_m_0)

candidate_m_0, m_0 == candidate_m_0,\
candidate_m_1, m_1 == candidate_m_1

(b'D\x9f]\xdd\x11\xeb\xb4\xec\x92iE\xfb.q1K>',
 False,
 b'D\x9f]\xdd\x11\xeb\xb4\xec\x92iE\xfa.q)S+',
 True)

In [27]:
decrypt(candidate_m_0, key=s_0), decrypt(candidate_m_1, key=s_0)

(b'subcircuit-0..qwe', b'subcircuit-1..iop')

In [28]:
decrypt(candidate_m_0, key=s_1), decrypt(candidate_m_1, key=s_1)

(b'\x00a>k\x0e\x08\x8b\x8d\xcc\x06\xd9\xdc1]}\x10:',
 b'\x00a>k\x0e\x08\x8b\x8d\xcc\x06\xd9\xdd1]e\x08/')